In [2]:
#standard Python libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from collections import OrderedDict
import glob
from PIL import Image
import numpy as np
import csv
#Weights and Biases - included for experiment tracking
#import wandb

#custom libraries included with project
from utils.RunBuilder import RunBuilder
from utils.RunManager import RunManager
from utils.ModelFactory import ModelFactory
from utils.DatasetFactory import DatasetFactory

In [3]:
#image input dimensions
image_input_width = 224
image_input_height = 224

#where to look for the training data
data_training_directory = "training_data/V1"

#where to look for the test images
data_test_directory = "test_data/"

#what model to use and wieghts to load
model_to_use = "resnet50"
model_weights = "models/BEST_V1-resnet50_dataset_squarepad_b64_lr0.0001_e5_5"


In [4]:
#make sure to pass the correct name for the dataset you used to train the model
transform = DatasetFactory.get_dataset('dataset_squarepad', data_training_directory, 1,  image_input_width, image_input_height, False)

In [6]:
dataset = datasets.ImageFolder(data_training_directory)

#Class Information from dataset
number_of_classes = len(dataset.classes)
print(number_of_classes)
class_names = dataset.classes
print(class_names)
print(type(class_names))

6
['Asterionellopsis', 'Chaetoceros', 'Dinophysis', 'Licmophora', 'Octactis', 'Pseudo-nitzschia']
<class 'list'>


In [7]:
#load the model architecture from the model factory
model = ModelFactory.get_network(model_to_use, number_of_classes)
#load the weights 
model.load_state_dict(torch.load(model_weights, weights_only=True))

#if we have CUDA then move the model to it
if torch.cuda.is_available():
      model.to('cuda')

#put the model in evaluation mode
model = model.eval()

Random seed set as 42


In [8]:
dataset = datasets.ImageFolder(data_training_directory)

#Class Information from dataset
number_of_classes = len(dataset.classes)
print(number_of_classes)
class_names = dataset.classes
print(class_names)
print(type(class_names))

6
['Asterionellopsis', 'Chaetoceros', 'Dinophysis', 'Licmophora', 'Octactis', 'Pseudo-nitzschia']
<class 'list'>


Classify a Single Image

In [31]:
file_name = data_test_directory + 'a_1834_D20240417T172618_IFCB172_t492_x756_y126_49198538.png'
print(file_name)
image = Image.open(file_name)

if image.mode != 'RGB':
    image = image.convert('RGB')

input_tensor = transform(image)
            
input_tensor = input_tensor.unsqueeze(0)

if torch.cuda.is_available():
    input_tensor = input_tensor.to('cuda')



#pass the image through the classifier    
with torch.inference_mode():

    output = model(input_tensor)

    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    print(probabilities)
    max_index = torch.argmax(probabilities)
    max_score = probabilities[max_index].item()
    string_prob_list = np.array2string(probabilities.cpu().numpy())
    print(max_score)       
    final_class = max_index.item()

    print(class_names[final_class])
    


test_data/a_1834_D20240417T172618_IFCB172_t492_x756_y126_49198538.png
tensor([1.2511e-05, 9.9998e-01, 2.6360e-07, 1.4631e-06, 5.3299e-07, 3.9717e-07],
       device='cuda:0')
0.999984860420227
Chaetoceros


In [9]:
import glob
image_files = glob.glob(data_test_directory + '*.png')



for file_name in image_files:
    print(file_name)
    image = Image.open(file_name)
    if image.mode != 'RGB':
        image = image.convert('RGB')

    input_tensor = transform(image)
                
    input_tensor = input_tensor.unsqueeze(0)

    if torch.cuda.is_available():
        input_tensor = input_tensor.to('cuda')



    #pass the image through the classifier    
    with torch.inference_mode():

        output = model(input_tensor)

        # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        max_index = torch.argmax(probabilities)
        max_score = probabilities[max_index].item()
        string_prob_list = np.array2string(probabilities.cpu().numpy())
        print(string_prob_list)       
        final_class = max_index.item()

        print(class_names[final_class]) 

        results = {'file_name': file_name, 'class_name' : class_names[final_class], 'score': max_score }
        with open(f'classifier_results.csv', 'a', newline='') as f:
            write = csv.DictWriter(f, fieldnames=['file_name','class_name', 'score'])
            if f.tell() == 0:
                write.writeheader()
            write.writerow(results)



test_data\a_1834_D20240417T172618_IFCB172_t492_x756_y126_49198538.png
[1.2510752e-05 9.9998486e-01 2.6360200e-07 1.4631314e-06 5.3298800e-07
 3.9717054e-07]
Chaetoceros
test_data\D20230423T231640_IFCB147_t588_x708_y310_6859236.png
[0.03090797 0.4660866  0.02643475 0.05153947 0.41014218 0.01488906]
Chaetoceros
test_data\D20230425T023612_IFCB147_t497_x732_y654_6914921.png
[4.5905490e-05 9.9984550e-01 3.2874837e-06 3.3679858e-05 5.5470897e-05
 1.6047585e-05]
Chaetoceros
test_data\D20230425T144531_IFCB147_t93_x676_y246_6948529.png
[5.3577232e-06 9.9998283e-01 4.4348113e-07 4.0111922e-06 2.0821126e-06
 5.2226578e-06]
Chaetoceros
test_data\D20230426T152909_IFCB147_t211_x684_y334_7031588.png
[2.4366867e-05 9.9993277e-01 2.8146451e-06 1.5552940e-05 1.1757094e-05
 1.2748703e-05]
Chaetoceros
test_data\D20230426T184833_IFCB147_t352_x692_y350_7039876.png
[2.2803442e-05 9.9993575e-01 2.6522264e-06 1.9622043e-05 8.9569767e-06
 1.0086882e-05]
Chaetoceros
test_data\D20230427T211200_IFCB147_t41_x748_y6